# Rasdaman Metadata Rodeo

The first section of this notebook explores the process of fetching coverage metadata from Rasdaman, and populated a "metadata catalog" for use in API request validation and coverage queries. The exploration here is limited to the coverages that are actually referenced in the API codebase.

The second section fetches coverage data in netCDF format, and shows the decoding behavior of `xarray` for different coverage data structures. Here we see how the ingest process may have downstream effects when using the coverage data in a programming context.

## 1. Fetching coverage metadata

### Set up functions to fetch relevant metadata

In [1]:
import requests
import json
import xml.etree.ElementTree as ET
import xarray as xr
import io
import os
import pandas as pd

In [2]:
def create_coverage_metadata_dict(ras_base_url):
    # get all coverage IDs in the rasdaman database
    get_capabilities_url = (
        f"{ras_base_url}/ows?&SERVICE=WCS&ACCEPTVERSIONS=2.1.0&REQUEST=GetCapabilities"
    )

    coverage_ids = {}

    with requests.get(get_capabilities_url) as response:
        if response.status_code == 200:
            # parse XML response to get list of coverage IDs from the nested XML
            root = ET.fromstring(response.text)
            for coverage_summary in root.findall(
                "{http://www.opengis.net/wcs/2.0}Contents/{http://www.opengis.net/wcs/2.0}CoverageSummary"
            ):
                coverage_id = coverage_summary.find(
                    "{http://www.opengis.net/wcs/2.0}CoverageId"
                ).text
                coverage_ids[coverage_id] = {}
        else:
            print(f"Failed to get capabilities: {response.status_code}")
            print(response.text)

    for coverage_id in coverage_ids:
        # get metadata for each coverage ID
        # print(f"Getting metadata for coverage ID: {coverage_id}")

        coverage_metadata = {}

        describe_coverage_url = f"{ras_base_url}/ows?&SERVICE=WCS&VERSION=2.1.0&REQUEST=DescribeCoverage&COVERAGEID={coverage_id}&outputType=GeneralGridCoverage"

        with requests.get(describe_coverage_url) as response:
            if response.status_code == 200:
                # parse XML response to get metadata for each coverage ID
                # we want axis labels & their ranges, the CRS, and the encodings dict

                root = ET.fromstring(response.text)
                for coverage_description in root.findall(
                    "{http://www.opengis.net/wcs/2.1/gml}CoverageDescription"
                ):

                    axis_info = {}
                    crs = ""
                    bands = []
                    range_types = []
                    encodings = []
                    file_refs = []

                    coverage_id = coverage_description.find(
                        "{http://www.opengis.net/wcs/2.0}CoverageId"
                    ).text

                    for envelope in coverage_description.findall(
                        "{http://www.opengis.net/cis/1.1/gml}Envelope"
                    ):
                        try:
                            crs = envelope.attrib["srsName"].split("crs/EPSG/0/")[1]
                        except:
                            crs = None

                        for axis_extent in envelope.findall(
                            "{http://www.opengis.net/cis/1.1/gml}AxisExtent"
                        ):
                            axis_info[axis_extent.attrib["axisLabel"]] = {
                                "lowerBound": axis_extent.attrib["lowerBound"],
                                "upperBound": axis_extent.attrib["upperBound"],
                            }

                    for metadata in coverage_description.findall(
                        "{http://www.opengis.net/cis/1.1/gml}Metadata"
                    ):
                        for cov_metadata in metadata.findall(
                            "{http://www.rasdaman.org}covMetadata"
                        ):
                            for slices in cov_metadata.findall(
                                "{http://www.rasdaman.org}slices"
                            ):
                                for slice in slices.findall(
                                    "{http://www.rasdaman.org}slice"
                                ):
                                    for encoding in slice.findall(
                                        "{http://www.rasdaman.org}Encoding"
                                    ):
                                        # drop any double quotes in the encoding string (sometimes entire value dict is wrapped in quotes)
                                        encoding.text = encoding.text.replace('"', "")
                                        # replace single quotes with double quotes to make it valid JSON
                                        encoding.text = encoding.text.replace("'", '"')
                                        # add to encodings list if not already there
                                        if encoding.text not in encodings:
                                            encodings.append(encoding.text)

                                    for file_ref in slice.findall(
                                        "{http://www.rasdaman.org}fileReferenceHistory"
                                    ):
                                        file_refs.append(file_ref.text)

                            for cov_band in cov_metadata.findall(
                                "{http://www.rasdaman.org}bands"
                            ):
                                cov_bands = list(cov_band)
                                for band in cov_bands:
                                    band_tag = band.tag
                                    bands.append(band_tag.split("}")[1])

                    for range_type in coverage_description.findall(
                        "{http://www.opengis.net/cis/1.1/gml}RangeType"
                    ):
                        for data_record in range_type.findall(
                            "{http://www.opengis.net/swe/2.0}DataRecord"
                        ):
                            for field in data_record.findall(
                                "{http://www.opengis.net/swe/2.0}field"
                            ):
                                range_types.append(field.attrib["name"])

                    coverage_metadata["axis_info"] = axis_info
                    coverage_metadata["crs"] = crs
                    coverage_metadata["bands"] = bands
                    coverage_metadata["range_types"] = range_types
                    coverage_metadata["encodings"] = encodings
                    coverage_metadata["file_refs"] = file_refs

                    coverage_ids[coverage_id] = coverage_metadata

            else:
                print(
                    f"Failed to get coverage metadata for {coverage_id}: {response.status_code}"
                )
                print(response.text)

    return coverage_ids

In [3]:
def search_api_codebase_for_coverage_ids(coverage_ids, api_codebase_path):
    coverage_id_found = []
    coverage_id_not_found = []

    for coverage_id in coverage_ids:
        coverage_id_found_flag = False
        for root, dirs, files in os.walk(api_codebase_path):
            for file in files:
                if file.endswith(".py"):
                    with open(os.path.join(root, file), "r") as f:
                        if coverage_id in f.read():
                            coverage_id_found_flag = True
                            break
            if coverage_id_found_flag:
                break

        if coverage_id_found_flag:
            coverage_id_found.append(coverage_id)
        else:
            coverage_id_not_found.append(coverage_id)

    return coverage_id_found, coverage_id_not_found

### Fetch coverage metadata

In [4]:
coverage_info = create_coverage_metadata_dict("https://zeus.snap.uaf.edu/rasdaman/")
coverage_ids_found, coverage_ids_not_found = search_api_codebase_for_coverage_ids(
    coverage_info.keys(), "/Users/joshpaul/data-api"
)

# drop coverage IDs that were not found in the API codebase from coverage info
for coverage_id in coverage_ids_not_found:
    del coverage_info[coverage_id]

print(f"{len(coverage_info)} coverage IDs found in API codebase:\n")

for key in coverage_info:
    print(key)

27 coverage IDs found in API codebase:

air_freezing_index_Fdays
air_thawing_index_Fdays
alfresco_relative_flammability_30yr
alfresco_vegetation_type_percentage
annual_mean_temp
annual_precip_totals_mm
ardac_beaufort_daily_slie
ardac_chukchi_daily_slie
beetle_risk
cmip6_indicators
cmip6_monthly
crrel_gipl_outputs
degree_days_below_zero_Fdays
dot_precip
heating_degree_days_Fdays
hsia_arctic_production
hydrology
iem_ar5_2km_taspr_seasonal
iem_cru_2km_taspr_seasonal
iem_cru_2km_taspr_seasonal_baseline_stats
jan_min_max_mean_temp
july_min_max_mean_temp
mean_annual_snowfall_mm
ncar12km_indicators_era_summaries
tas_2km_historical
tas_2km_projected
wet_days_per_year


### Summarize the metadata

In [5]:
# for each coverage in the dict, find all unique axis names, band names, range types, and CRS's
unique_axis_names = set()
unique_bands = set()
unique_range_types = set()
unique_band_and_range_types = set()
unique_crs = set()

for coverage_id in coverage_info:
    for axis_name in coverage_info[coverage_id]["axis_info"]:
        unique_axis_names.add(axis_name)
    for band in coverage_info[coverage_id]["bands"]:
        unique_bands.add(band)
        unique_band_and_range_types.add(band)
    for range_type in coverage_info[coverage_id]["range_types"]:
        unique_range_types.add(range_type)
        unique_band_and_range_types.add(range_type)
    unique_crs.add(coverage_info[coverage_id]["crs"])

print("Unique axis names:")
print(unique_axis_names)
print("\n")

print("Unique bands:")
print(unique_bands)
print("\n")

print("Unique range types:")
print(unique_range_types)
print("\n")

print("Unique CRS's:")
print(unique_crs)
print("\n")

print("Unique bands + range types:")
print(unique_band_and_range_types)
print("\n")

Unique axis names:
{'tempstat', 'model', 'year', 'ansi', 'scenario', 'stat', 'duration', 'season', 'X', 'veg_type', 'variable', 'month', 'era', 'varname', 'snowpack', 'decade', 'indicator', 'Y', 'interval', 'lat', 'lon'}


Unique bands:
{'sm3', 'swe', 'iwe', 'pcp', 'tmax', 'evap', 'slie', 'snow_melt', 'glacier_melt', 'sm1', 'runoff', 'data', 'tasmax', 'pf_upper', 'dw', 'pf', 'tas', 'rx1day', 'tmin', 'su', 'ftc', 'pf_lower', 'sm2', 'tasmin'}


Unique range types:
{'sm3', 'swe', 'iwe', 'pcp', 'tmax', 'evap', 'slie', 'air_thawing_index_Fdays', 'glacier_melt', 'snow_melt', 'sm1', 'Gray', 'runoff', 'data', 'degree_days_below_zero_Fdays', 'tasmax', 'pf_upper', 'heating_degree_days_Fdays', 'dw', 'pf', 'tas', 'rx1day', 'tmin', 'su', 'ftc', 'pf_lower', 'air_freezing_index_Fdays', 'sm2', 'tasmin'}


Unique CRS's:
{'3572', '4326', '3338'}


Unique bands + range types:
{'sm3', 'swe', 'iwe', 'pcp', 'tmax', 'evap', 'slie', 'air_thawing_index_Fdays', 'glacier_melt', 'snow_melt', 'sm1', 'Gray', 'runof

In [6]:
# count reference files
for coverage_id in coverage_info:
    no_ref_files = len(coverage_info[coverage_id]["file_refs"])
    coverage_info[coverage_id]["no_ref_files"] = no_ref_files
    if no_ref_files != 0:
        coverage_info[coverage_id]["file_type"] = (
            coverage_info[coverage_id]["file_refs"][0].split("/")[-1].split(".")[-1]
        )
    else:
        coverage_info[coverage_id]["file_type"] = None

# create a dataframe with the coverage id, number of reference files, file type,
# and a flag if any axis names that are not in the nonvar_axes list
# and a flag if "gray" is in the range types
nonvar_axes = [
    "model",
    "scenario",
    "era",
    "season",
    "ansi",
    "month",
    "year",
    "time",
    "decade",
    "geom_id",
    "lat",
    "Y",
    "lon",
    "X",
]

coverage_df = pd.DataFrame.from_dict(coverage_info, orient="index")
coverage_df = coverage_df.reset_index()
coverage_df = coverage_df.rename(columns={"index": "coverage_id"})
coverage_df["var_axes"] = coverage_df["axis_info"].apply(
    lambda x: [axis for axis in x.keys() if axis not in nonvar_axes]
)
coverage_df["gray_range_type"] = coverage_df["range_types"].apply(lambda x: "Gray" in x)

coverage_df = coverage_df[
    ["coverage_id", "no_ref_files", "file_type", "var_axes", "gray_range_type"]
]

coverage_df

,coverage_id,no_ref_files,file_type,var_axes,gray_range_type
0,air_freezing_index_Fdays,1,nc,[],False
1,air_thawing_index_Fdays,1,nc,[],False
2,alfresco_relative_flammability_30yr,56,tif,[],True
3,alfresco_vegetation_type_percentage,414,tif,[veg_type],True
4,annual_mean_temp,1825,tif,[],True
5,annual_precip_totals_mm,1825,tif,[],True
6,ardac_beaufort_daily_slie,0,None,[],False
7,ardac_chukchi_daily_slie,0,None,[],False
8,beetle_risk,50,tif,[snowpack],True
9,cmip6_indicators,1,nc,[],False


### Export metadata as JSON file
This is a mockup of a "metadata catalog" that could be used for request validation and general query of our entire collection of coverages. Some fields here are added for demo purposes, but would ideally be populated in the coverage metadata and pulled from Rasdaman. 

Note that in it's current state, this JSON would be difficult to search across coverages for presence / absence of specific variables, or to search across coverages for a specific time period. Standardizing the coverage structure would help with this.

In [7]:
# remove the file_refs key from the coverage_info dict
# and only use the first encoding from the list
coverage_info_no_refs = {}
for coverage_id in coverage_info:
    coverage_info_no_refs[coverage_id] = coverage_info[coverage_id].copy()
    del coverage_info_no_refs[coverage_id]["file_refs"]
    if coverage_info[coverage_id]["encodings"] != []:
        coverage_info_no_refs[coverage_id]["encodings"] = json.loads(
            coverage_info[coverage_id]["encodings"][0]
        )
    else:
        coverage_info_no_refs[coverage_id]["encodings"] = None

    # add placeholder dict for data source
    coverage_info_no_refs[coverage_id]["data_source"] = {
        "title": None,
        "date": None,
        "url": None,
        "doi": None,
        "authors": [],
    }

    # add some SNAP data processing info
    coverage_info_no_refs[coverage_id]["data_processing"] = {
        "authors": ["Scenarios Network for Alaska and Arctic Planning (SNAP)"],
        "date": None,
        "contact": "uaf-snap-data-tools@alaska.edu",
        "notes": None,
    }

# write the coverage_info dict to a JSON file
coverage_info_json = json.dumps(coverage_info_no_refs, indent=4)
coverage_info_json_file = "coverage_metadata.json"
with open(coverage_info_json_file, "w") as f:
    f.write(coverage_info_json)

## 2. How are coverages decoded by `xarray` ?

### Situation 1:
A coverage has variables stored as bands. When read by `xarray`, these automatically show up as data variables, with the band metadata intact.
There is only one "slice" in the coverage metadata, and this contains the single encodings dictionary which is populated as a dataset "Encoding" attribute.

In [8]:
# cmip6_indicators coverage has variables stored as "bands"
# see coverage metadata here: https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.1.0&REQUEST=DescribeCoverage&COVERAGEID=cmip6_indicators&outputType=GeneralGridCoverage
url = "https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=cmip6_indicators&SUBSET=year(1950,1951)&FORMAT=application/netcdf"

r = requests.get(url)
data = r.content
ds = xr.open_dataset(io.BytesIO(data))

ds

<xarray.Dataset> Size: 48MB
Dimensions:   (scenario: 5, model: 12, year: 2, lon: 288, lat: 43)
Coordinates:
  * scenario  (scenario) float64 40B 0.0 1.0 2.0 3.0 4.0
  * model     (model) float64 96B 0.0 1.0 2.0 3.0 4.0 ... 7.0 8.0 9.0 10.0 11.0
  * year      (year) float64 16B 1.95e+03 1.951e+03
  * lon       (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
  * lat       (lat) float64 344B 90.0 89.06 88.12 87.17 ... 52.3 51.36 50.42
Data variables:
    crs       |S1 1B ...
    dw        (scenario, model, year, lon, lat) float64 12MB ...
    ftc       (scenario, model, year, lon, lat) float64 12MB ...
    rx1day    (scenario, model, year, lon, lat) float64 12MB ...
    su        (scenario, model, year, lon, lat) float64 12MB ...
Attributes:
    Encoding:              {'model': {'0': 'CESM2', '1': 'CNRM-CM6-1-HR', '2'...
    Title:                 CMIP6 Indicators
    fileReferenceHistory:  /opt/rasdaman-storage/user_data/snapdata/rasdaman-...

### Situation 2:
A coverage has variables stored as range type. When read by `xarray`, the range types automatically show up as data variables, but with no metadata to describe what the values mean. There are many "slice" combinations in the coverage metadata, and these each contains an encodings dictionary which are all redundantly populated in the dataset "Encodings" attribute.

In [9]:
# design_freezing_index coverage has variables stored as "range types", in this case labeled as "Gray"
# see coverage metadata here: https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.1.0&REQUEST=DescribeCoverage&COVERAGEID=design_freezing_index&outputType=GeneralGridCoverage
url = "https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=design_freezing_index&SUBSET=X(-1000000,1000000)&SUBSET=Y(1000000,2000000)&FORMAT=application/netcdf"

r = requests.get(url)
data = r.content
ds = xr.open_dataset(io.BytesIO(data))

ds

<xarray.Dataset> Size: 217kB
Dimensions:                    (model: 3, era: 3, X: 109, Y: 55)
Coordinates:
  * model                      (model) float64 24B 0.0 1.0 2.0
  * era                        (era) float64 24B 0.0 1.0 2.0
  * X                          (X) float64 872B -9.946e+05 ... 1.002e+06
  * Y                          (Y) float64 440B 1.997e+06 ... 9.984e+05
Data variables:
    albers_conical_equal_area  |S1 1B ...
    Gray                       (model, era, X, Y) float32 216kB ...
Attributes:
    Encoding:              {'model': {'1': 'GFDL-CM3', '0': 'ERA-Interim', '2...
    Title:                 Design Freezing Index
    fileReferenceHistory:  /opt/rasdaman-storage/user_data/rltorgerson/rasdam...

### Scenario 3:
A coverage has variables stored range type, and accessed through axes (dimensions). When read by `xarray`, the range types automatically show up as data variables, but with no metadata to describe what the values mean. There are many "slice" combinations in the coverage metadata, and these each contains an encodings dictionary which are all redundantly populated in the dataset "Encodings" attribute.

In [10]:
# the iem_ar5_2km_tasper_seasonal coverage has variables stored as "range types", in this case labeled as "Gray" and accessed by axis combinations
# see coverage metadata here: https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.1.0&REQUEST=DescribeCoverage&COVERAGEID=iem_ar5_2km_taspr_seasonal&outputType=GeneralGridCoverage

url = "https://zeus.snap.uaf.edu/rasdaman/ows?&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=iem_ar5_2km_taspr_seasonal&SUBSET=X(-100000,100000)&SUBSET=Y(500000,600000)&FORMAT=application/netcdf"

r = requests.get(url)
data = r.content
ds = xr.open_dataset(io.BytesIO(data))

ds

<xarray.Dataset> Size: 13MB
Dimensions:                    (decade: 9, season: 4, model: 3, scenario: 3,
                                varname: 2, X: 101, Y: 51)
Coordinates:
  * decade                     (decade) float64 72B 0.0 1.0 2.0 ... 6.0 7.0 8.0
  * season                     (season) float64 32B 0.0 1.0 2.0 3.0
  * model                      (model) float64 24B 0.0 1.0 2.0
  * scenario                   (scenario) float64 24B 0.0 1.0 2.0
  * varname                    (varname) float64 16B 0.0 1.0
  * X                          (X) float64 808B -1.002e+05 ... 9.978e+04
  * Y                          (Y) float64 408B 5.994e+05 ... 4.994e+05
Data variables:
    albers_conical_equal_area  |S1 1B ...
    Gray                       (decade, season, model, scenario, varname, X, Y) float32 13MB ...
Attributes:
    Encoding:              {'decade': {'0': '2010_2019', '1': '2020_2029', '2...
    Title:                 Seasonal-decadal summaries of AR5 temperature and ...
    fileReferenceHistory:  /opt/rasdaman-storage/user_data/snapdata/rasdaman-...